In [1]:
import torch 
import torch.autograd as autograd
import numpy as np 
import torch.nn as nn
from load_data import *
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


In [2]:
## Loading data 

embedding_dict = load_embedding_dict()
sentence_Y_dict = load_sentence_Y_dict()

print "Loading batch dicts"
train_batch_dict_new = load_train_data()
validation_batch_dict_new = load_validation_data()


Loading batch dicts


In [3]:
corpus_word_vocab = load_corpus_vocab()

In [4]:
vocab_size = len(corpus_word_vocab)
embedding_dim = 300
hidden_dim = 100
batch_size = 64 
n_classes = 5

In [5]:
embedding_lookup  = nn.Embedding(vocab_size,embedding_dim,padding_idx = 0)

In [6]:
bilstm = nn.LSTM(embedding_dim,hidden_dim, bidirectional = True)
linear_layer = nn.Linear(2*hidden_dim,n_classes)


In [12]:
class BiLSTM(nn.Module) :
    
    def __init__(self,embedding_dim,hidden_dim,vocab_size,n_classes) :
        
        super(BiLSTM,self).__init__()
        self.embedding_dim = embedding_dim 
        self.hidden_dim = hidden_dim 
        self.vocab_size = vocab_size
        self.n_classes = n_classes
        
        self.embedding_lookup = nn.Embedding(vocab_size,embedding_dim,padding_idx = 0)
        self.bilstm = nn.LSTM(embedding_dim,hidden_dim, bidirectional = True)
        self.linear_layer = nn.Linear(2*hidden_dim,n_classes)
        self.hidden = self.init_hidden(hidden_dim)
        
        
    def forward(self,sentence,embedding_dict,s_id) :
        
        embeds = self.embedding_lookup(sentence)
       
        #print "embedding dimension",embeds.view(len(embedding_dict[s_id]),1,embedding_dim).size()

        lstm_out, self.hidden = self.bilstm(embeds.view(len(embedding_dict[s_id]),1,embedding_dim),self.hidden)
        lstm_out = lstm_out.view(len(embedding_dict[s_id]),2*hidden_dim)

        output_vec = self.linear_layer(lstm_out)

        return output_vec
    def init_hidden(self,hidden_dim) :
        return (Variable(torch.randn(2,1,hidden_dim)),Variable(torch.randn(2,1,hidden_dim)))
        
        

In [7]:
def convert_to_onehot(batch_labels) :


	### Label -1 assigned to the pad values and before input and mapped  to [0 0 0 0 0]
	output_mat = []
	for sent_labels in batch_labels :
		temp_mat = []	
		for word_label in sent_labels :
			if word_label == -1:

				temp = [0]*5
				temp_mat.append(temp)
				continue
			temp = [0]*5
			temp[word_label] = 1
			temp_mat.append(temp)

		output_mat.append(temp_mat)

	return np.asarray(output_mat)

def pad_zeros(ids,embedding_dict) :
	## return a numpy array of all the sentences in the batch padded with zeros for their embeddings 

	max_len = 0 
	for sent_id in ids :
		embedding = embedding_dict[sent_id]
		if len(embedding) > max_len :
			max_len = len(embedding)

	#print "max_len", max_len


	sent_mat = [embedding_dict[sent_id] for sent_id in ids]

	if len(np.array(sent_mat).shape) == 1 :
			return np.array([xi + [0]*(max_len - len(xi)) for xi in sent_mat],dtype='int'),max_len

	else :

		padded_mat = np.zeros((len(ids),max_len),dtype='int')

		sent_mat = np.array(sent_mat,dtype= 'int')
		for i in xrange(len(ids)) :
	 		padded_mat[i,:] = np.pad(sent_mat[i,:],(0,max_len - sent_mat[i,:].shape[0]),'constant',constant_values = (0))


		return padded_mat,max_len

def test_on_validation_data(model,sentence_Y_dict, validation_batch_dict_new,embedding_dict,use_whole = False) :

	batch_count = 0 
	validation_loss = 0 
	acc  = 0 
	for batch_n,sent_ids in validation_batch_dict_new.iteritems() :

		vList = []
		batch_count +=1 

		sents , max_len = pad_zeros(sent_ids,embedding_dict)

		labels = [sentence_Y_dict[item]+[-1]*(max_len - len(sentence_Y_dict[item])) for item in sent_ids] 
				
		labels = convert_to_onehot(labels)

		vList = model.test_on_batch(sents,labels)
		validation_loss+= vList[0]
		acc+=vList[1]

		if batch_count == 30 and use_whole == False :
			break 


	print "validation_loss :",validation_loss/batch_count
	acc = acc/batch_count 
	print "Accuracy :",acc
	return [validation_loss/batch_count,acc]


In [28]:
## Preparing data for input 
epochs = 1
loss_function = nn.CrossEntropyLoss()
## Going over each sentence at a time 

batch_size = 64

model = BiLSTM(embedding_dim,hidden_dim,vocab_size,n_classes)
optimizer = torch.optim.RMSprop(model.parameters(),lr = 0.0001)
def intialise_hidden(hidden_dim) :
    
    return (Variable(torch.randn(2,1,hidden_dim)),Variable(torch.randn(2,1,hidden_dim)))

for i in range(epochs) :
    batch_count = 0
    
    for batch,sent_ids in train_batch_dict_new.iteritems() :
        batch_count+= 1
        s_count = 0 
        
        sent_loss = Variable(torch.FloatTensor(len(sent_ids))) ## len(sent_ids)
        for s_id in sent_ids :
            
            model.zero_grad()
            #hidden = intialise_hidden(hidden_dim)
            
            model.hidden = model.init_hidden(hidden_dim)
            
            sentence = Variable(torch.LongTensor(embedding_dict[s_id]))
            labels = sentence_Y_dict[s_id]

            targets = Variable(torch.LongTensor(labels))
            #print "target_size", targets.size()
            
            output_vec = model.forward(sentence,embedding_dict,s_id)
            """embeds = embedding_lookup(sentence)
       
            #print "embedding dimension",embeds.view(len(embedding_dict[s_id]),1,embedding_dim).size()
        
            lstm_out, hidden = bilstm(embeds.view(len(embedding_dict[s_id]),1,embedding_dim),hidden)
            lstm_out = lstm_out.view(len(embedding_dict[s_id]),2*hidden_dim)

            output_vec = linear_layer(lstm_out)
            """
            # output_vec --> sentence_length * n_classes
            
            
            #if s_count == 10 :
                #break 
            sent_loss[s_count] = loss_function(output_vec,targets)
            s_count+=1 
            
        ## One batch finished 
        avg_batch_loss = sum(sent_loss)/batch_size
        print avg_batch_loss
        avg_batch_loss.backward()
        optimizer.step()
        
        if batch_count ==10 :
            break 

Variable containing:
 1.5908
[torch.FloatTensor of size 1]

Variable containing:
 1.5291
[torch.FloatTensor of size 1]

Variable containing:
 1.4914
[torch.FloatTensor of size 1]

Variable containing:
 1.4479
[torch.FloatTensor of size 1]

Variable containing:
 1.4303
[torch.FloatTensor of size 1]

Variable containing:
 1.3929
[torch.FloatTensor of size 1]

Variable containing:
 1.3493
[torch.FloatTensor of size 1]

Variable containing:
 1.3313
[torch.FloatTensor of size 1]

Variable containing:
 1.2973
[torch.FloatTensor of size 1]

Variable containing:
 1.2944
[torch.FloatTensor of size 1]



In [31]:
model.parameters.data

AttributeError: 'function' object has no attribute 'data'

In [137]:
batch_loss

Variable containing:
 1.5970
 1.6201
 1.5854
 1.6064
 1.6107
 1.6083
 1.6089
 1.5606
 1.6235
 1.5747
[torch.FloatTensor of size 10]